**V2 Testing**
# Critical implementation bugs (fix first)


1. pydicom API misuse

- AttributeError: module 'pydicom' has no attribute 'read_file'. Use the supported API: pydicom.dcmread(...) (older read_file deprecations).
- Verify pydicom version and use dcmread. Also use .pixel_array only after confirming hasattr.

2. Random projection inside forward pass (major bug)

- tab_proj = F.linear(tab_expanded, torch.randn(1024, 512).to(images.device))
- You are using a new random matrix every forward pass. That destroys learning of cross-modal attention. Replace with a trainable nn.Linear(512, 1024) (initialized once). This is probably the single biggest structural bug.

3. Inconsistent sigma / scale handling and arbitrary multipliers

- In training you clamp sigma = exp(log_var/2) to min 2.0 — while evaluation uses sigma floor 70.0. In submission you set confidence_val = max(exp(log_var/2) * 70, 70) (why multiply by 70?). This mismatch creates meaningless confidences and ruins LLL.
- Fix: keep internal sigma in natural units of FVC; do not multiply by arbitrary constants. Only apply the contest evaluation floor (70) at scoring time — not during training.

4. Submission pipeline clipping hides bug

- You clamp predictions to [800,6000] during submission — all predictions hitting 800 likely reflect upstream underprediction or wrong scale. Remove clipping while debugging; investigate raw predictions distribution first.

5. Hard-coded patient exclusions & path issues

- Filtering out two patient IDs silently in dataset init is suspicious. Document why or remove. Ensure mapping patient → image files is correct and complete.

6. ModelWithConfidence referenced but not defined → NameError later. Keep code consistent.

In [4]:
# Install required packages
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '--quiet'])
        return True
    except subprocess.CalledProcessError:
        return False

print("Installing required packages...")
packages = [
    'pydicom',
    'pylibjpeg',
    'pylibjpeg-libjpeg', 
    'gdcm',
    'opencv-python-headless',
    'scikit-learn',
    'albumentations',
    'tqdm',
    'seaborn'
]

for pkg in packages:
    if install_package(pkg):
        print(f"✅ {pkg} installed")
    else:
        print(f"⚠️ {pkg} installation failed (may already be installed)")


Installing required packages...
✅ pydicom installed
✅ pylibjpeg installed
✅ pylibjpeg-libjpeg installed
✅ gdcm installed
✅ opencv-python-headless installed
✅ scikit-learn installed
✅ albumentations installed
✅ tqdm installed
✅ seaborn installed


In [5]:
# Import libraries
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import random
from tqdm import tqdm 
from datetime import timedelta, datetime
from pathlib import Path
import json
import warnings
import pickle
import glob
from math import sqrt, log
from sklearn.metrics import mean_squared_error, r2_score

# Image processing
from skimage import measure, morphology, segmentation
from skimage.transform import resize
from scipy.ndimage import binary_dilation, binary_erosion
from skimage.measure import label, regionprops
from sklearn.cluster import KMeans
from skimage.segmentation import clear_border

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torch.cuda.amp import autocast, GradScaler

# Albumentations for medical augmentations
import albumentations as albu
from albumentations.pytorch import ToTensorV2

# Model selection
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import lightgbm as lgb

# Check for TPU/GPU acceleration
try:
    import torch_xla
    import torch_xla.core.xla_model as xm
    HAS_TPU = True
except ImportError:
    HAS_TPU = False

warnings.filterwarnings('ignore')

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(42)

# Configuration - Detect and use best available accelerator
if HAS_TPU:
    DEVICE = xm.xla_device()
    print("Using TPU accelerator")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU accelerator")
else:
    DEVICE = torch.device("cpu")
    print("Using CPU")

DATA_DIR = Path("../input/osic-pulmonary-fibrosis-progression")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

print("Pulmonary Fibrosis Progression Analysis - OPTIMIZED VERSION")
print(f"Device: {DEVICE}")

# =============================================================================
# PART 1: DATA LOADING AND EDA
# =============================================================================

# Load datasets
train_df = pd.read_csv(DATA_DIR / 'train.csv')
try:
    test_df = pd.read_csv(DATA_DIR / 'test.csv')
    print(f'Train: {train_df.shape[0]} rows, Test: {test_df.shape[0]} rows')
except:
    print(f'Train: {train_df.shape[0]} rows, Test: file not found')
    test_df = None

print("\nTrain data sample:")
print(train_df.head())

print("\nTrain data statistics:")
print(train_df.describe())

# Basic EDA
print(f'\nUnique patients in training data: {train_df["Patient"].nunique()}')
print(f'Total observations: {len(train_df)}')
print(f'Average observations per patient: {len(train_df)/train_df["Patient"].nunique():.2f}')

# Check for missing values
print(f'\nMissing values:')
print(train_df.isnull().sum())

# =============================================================================
# PART 2: FEATURE ENGINEERING
# =============================================================================

print("Processing tabular features...")

# Create baseline features for each patient
baseline_features = {}
patient_slopes = {}
patient_intercepts = {}

for patient in train_df['Patient'].unique():
    patient_data = train_df[train_df['Patient'] == patient].copy().sort_values('Weeks')
    
    # Get baseline measurement (first visit)
    baseline = patient_data.iloc[0]
    baseline_features[patient] = {
        'Age': baseline['Age'],
        'Sex': baseline['Sex'],
        'SmokingStatus': baseline['SmokingStatus'],
        'BaselineFVC': baseline['FVC'],
        'BaselineWeeks': baseline['Weeks'],
        'Percent': baseline['Percent'] if 'Percent' in baseline else 50.0  # Fallback
    }
    
    # Calculate slope and intercept if multiple measurements
    if len(patient_data) > 1:
        weeks = patient_data['Weeks'].values
        fvc = patient_data['FVC'].values
        
        # Linear regression: FVC = slope * weeks + intercept
        A = np.vstack([weeks, np.ones(len(weeks))]).T
        slope, intercept = np.linalg.lstsq(A, fvc, rcond=None)[0]
        
        patient_slopes[patient] = slope
        patient_intercepts[patient] = intercept
    else:
        patient_slopes[patient] = 0.0
        patient_intercepts[patient] = baseline['FVC']

# Create enhanced tabular features
def get_enhanced_tabular_features(patient_id, row=None):
    """Get enhanced tabular features with proper encoding"""
    features = baseline_features[patient_id].copy()
    
    # Standardize age
    features['Age'] = (features['Age'] - 50) / 20  # Standardize around mean
    
    # Encode sex (0 for Male, 1 for Female)
    features['Sex'] = 1 if features['Sex'] == 'Female' else 0
    
    # One-hot encode smoking status
    smoking_status = features['SmokingStatus']
    features['Smoking_Never'] = 1 if smoking_status == 'Never smoked' else 0
    features['Smoking_Ex'] = 1 if smoking_status == 'Ex-smoker' else 0
    features['Smoking_Current'] = 1 if smoking_status == 'Currently smokes' else 0
    
    # Standardize baseline FVC
    features['BaselineFVC'] = (features['BaselineFVC'] - 2500) / 1000
    
    # Standardize baseline weeks
    features['BaselineWeeks'] = features['BaselineWeeks'] / 100
    
    # Standardize Percent
    features['Percent'] = (features['Percent'] - 50) / 20
    
    # Add slope and intercept
    features['Slope'] = patient_slopes[patient_id] / 10  # Scale slope
    features['Intercept'] = (patient_intercepts[patient_id] - 2500) / 1000
    
    # Remove the original smoking status
    del features['SmokingStatus']
    
    # If we have a row with current week, add week delta
    if row is not None:
        week_delta = (row['Weeks'] - features['BaselineWeeks'] * 100) / 50
        features['WeekDelta'] = week_delta
    
    return np.array(list(features.values()), dtype=np.float32)

# Calculate slopes for each patient
A = patient_slopes
TAB = {patient: get_enhanced_tabular_features(patient) for patient in baseline_features.keys()}
P = list(baseline_features.keys())

print(f"Processed {len(P)} patients with enhanced features")

# =============================================================================
# PART 3: ENHANCED DICOM PROCESSING WITH HU PRESERVATION
# =============================================================================

def get_pixels_hu(dcm):
    """Convert DICOM pixel array to Hounsfield Units"""
    try:
        # Get pixel array
        pixel_array = dcm.pixel_array.astype(np.float32)
        
        # Apply rescale intercept and slope if available
        intercept = getattr(dcm, 'RescaleIntercept', 0)
        slope = getattr(dcm, 'RescaleSlope', 1)
        
        pixel_array = pixel_array * slope + intercept
        
        return pixel_array
    except:
        return np.zeros((512, 512), dtype=np.float32)

def load_dicom_with_hu(path):
    """Load DICOM and preserve Hounsfield Units"""
    try:
        dcm = pydicom.dcmread(str(path), force=True)
        hu_image = get_pixels_hu(dcm)
        
        # Window to lung range [-1200, 600]
        hu_image = np.clip(hu_image, -1200, 600)
        
        # Normalize to [-1, 1]
        hu_image = (hu_image + 300) / 900  # Center around -300, scale by 900
        
        return hu_image
    except Exception as e:
        return np.zeros((512, 512), dtype=np.float32) - 1  # Return -1 filled array

def load_three_slices(patient_dir, slice_idx=None, target_size=(256, 256)):
    """Load three adjacent slices for 2.5D input with consistent sizing"""
    try:
        # Get all DICOM files for patient
        dicom_files = sorted(list(patient_dir.glob('*.dcm')))
        if not dicom_files:
            return None
            
        # Use middle slice if not specified
        if slice_idx is None:
            slice_idx = len(dicom_files) // 2
            
        # Get three slices around the index
        slices = []
        for i in range(max(0, slice_idx-1), min(len(dicom_files), slice_idx+2)):
            slice_img = load_dicom_with_hu(dicom_files[i])
            
            # Resize to target size
            if slice_img.shape != target_size:
                slice_img = cv2.resize(slice_img, target_size, interpolation=cv2.INTER_AREA)
                
            slices.append(slice_img)
            
        # If we couldn't get 3 slices, duplicate existing ones
        while len(slices) < 3:
            slices.append(slices[-1] if slices else np.zeros(target_size, dtype=np.float32) - 1)
            
        # Stack slices as channels
        stacked = np.stack(slices, axis=-1)
        return stacked
        
    except Exception as e:
        # Return dummy 3-slice image with correct size
        dummy_slice = np.zeros(target_size, dtype=np.float32) - 1
        return np.stack([dummy_slice] * 3, axis=-1)

# =============================================================================
# PART 4: MEDICAL AUGMENTATIONS FOR HU IMAGES
# =============================================================================

class MedicalAugmentation:
    def __init__(self, augment=True, target_size=(256, 256)):
        self.target_size = target_size
        if augment:
            self.transform = albu.Compose([
                albu.Rotate(limit=5, p=0.3),  # Reduced rotation for medical images
                albu.HorizontalFlip(p=0.3),
                albu.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.05, rotate_limit=5, p=0.3),
                albu.GaussNoise(var_limit=(0.01, 0.05), p=0.2),  # Reduced noise for HU
                albu.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
                ToTensorV2()
            ])
        else:
            self.transform = albu.Compose([
                ToTensorV2()
            ])
    
    def __call__(self, image):
        # Ensure image is the correct size
        if image.shape[:2] != self.target_size:
            image = cv2.resize(image, self.target_size, interpolation=cv2.INTER_AREA)
        return self.transform(image=image)['image']

# =============================================================================
# PART 5: DATASET CLASS WITH 2.5D INPUT
# =============================================================================

class OSICDenseNetDataset(Dataset):
    """Dataset class with 2.5D input and robust error handling"""
    
    def __init__(self, patients, A_dict, TAB_dict, data_dir, split='train', augment=True, target_size=(256, 256)):
        self.patients = patients
        self.A_dict = A_dict
        self.TAB_dict = TAB_dict
        self.data_dir = Path(data_dir)
        self.split = split
        self.augment = augment
        self.target_size = target_size
        self.augmentor = MedicalAugmentation(augment=augment, target_size=target_size)
        
        # Preload patient directories and file lists
        self.patient_dirs = {}
        valid_patients = []
        
        for patient in self.patients:
            patient_dir = self.data_dir / patient
            
            if not patient_dir.exists():
                continue
                
            dicom_files = list(patient_dir.glob('*.dcm'))
            
            if dicom_files:
                self.patient_dirs[patient] = patient_dir
                valid_patients.append(patient)
        
        self.valid_patients = valid_patients
        print(f"Dataset {split}: {len(self.valid_patients)} valid patients")
    
    def __len__(self):
        return len(self.valid_patients) * (4 if self.split == 'train' else 1)
    
    def __getitem__(self, idx):
        try:
            patient_idx = idx % len(self.valid_patients)
            patient = self.valid_patients[patient_idx]
            
            # Load 2.5D image (3 slices)
            img = load_three_slices(self.patient_dirs[patient], target_size=self.target_size)
            
            # Apply augmentations
            img_tensor = self.augmentor(img)
            
            # Get features
            tab_features = torch.tensor(self.TAB_dict[patient], dtype=torch.float32)
            target = torch.tensor(self.A_dict[patient], dtype=torch.float32)
            
            return img_tensor, tab_features, target, patient
            
        except Exception as e:
            # Return dummy data with consistent sizes
            dummy_img = torch.zeros((3, self.target_size[0], self.target_size[1]), dtype=torch.float32)
            dummy_tab = torch.zeros(len(self.TAB_dict[patient]), dtype=torch.float32)
            dummy_target = torch.tensor(0.0, dtype=torch.float32)
            return dummy_img, dummy_tab, dummy_target, "dummy_patient"

# =============================================================================
# PART 6: IMPROVED MODEL ARCHITECTURE
# =============================================================================

class EfficientNetModel(nn.Module):
    """More efficient model using EfficientNet backbone"""
    
    def __init__(self, tabular_dim=10):
        super(EfficientNetModel, self).__init__()
        
        # EfficientNet backbone
        self.backbone = models.efficientnet_b0(pretrained=True)
        
        # Modify first convolution to accept 3 channels properly
        original_first_conv = self.backbone.features[0][0]
        self.backbone.features[0][0] = nn.Conv2d(
            3, original_first_conv.out_channels, 
            kernel_size=original_first_conv.kernel_size,
            stride=original_first_conv.stride,
            padding=original_first_conv.padding,
            bias=original_first_conv.bias
        )
        
        # Initialize with pretrained weights (average across RGB channels)
        with torch.no_grad():
            self.backbone.features[0][0].weight[:, :3] = original_first_conv.weight.clone()
            if original_first_conv.bias is not None:
                self.backbone.features[0][0].bias = original_first_conv.bias.clone()
        
        # Get number of features from backbone
        self.num_image_features = self.backbone.classifier[1].in_features
        
        # Tabular processor
        self.tabular_processor = nn.Sequential(
            nn.Linear(tabular_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # Fusion and prediction
        self.fusion = nn.Sequential(
            nn.Linear(self.num_image_features + 128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        self.mean_head = nn.Linear(128, 1)
        self.log_var_head = nn.Linear(128, 1)
        
    def forward(self, images, tabular):
        # Image features
        img_features = self.backbone.features(images)
        img_features = F.adaptive_avg_pool2d(img_features, (1, 1))
        img_features = img_features.view(img_features.size(0), -1)
        
        # Tabular features
        tab_features = self.tabular_processor(tabular)
        
        # Fusion
        combined = torch.cat([img_features, tab_features], dim=1)
        fused = self.fusion(combined)
        
        # Predictions
        mean_pred = self.mean_head(fused).squeeze(-1)
        log_var = self.log_var_head(fused).squeeze(-1)
        
        return mean_pred, log_var

# Initialize model
tabular_dim = len(TAB[list(TAB.keys())[0]])
model = EfficientNetModel(tabular_dim=tabular_dim).to(DEVICE)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# =============================================================================
# PART 7: DATA SPLIT AND LOADERS
# =============================================================================

# Split patients using GroupKFold for better validation
patients_list = list(P)
kfold = GroupKFold(n_splits=5)

# For simplicity, use the first fold
train_idx, val_idx = next(kfold.split(patients_list, groups=patients_list))
train_patients = [patients_list[i] for i in train_idx]
val_patients = [patients_list[i] for i in val_idx]

print(f"Train patients: {len(train_patients)}, Validation patients: {len(val_patients)}")

# Create datasets with consistent target size (smaller for faster training)
TARGET_SIZE = (256, 256)
train_dataset = OSICDenseNetDataset(
    patients=train_patients, A_dict=A, TAB_dict=TAB, 
    data_dir=TRAIN_DIR, split='train', augment=True, target_size=TARGET_SIZE
)

val_dataset = OSICDenseNetDataset(
    patients=val_patients, A_dict=A, TAB_dict=TAB,
    data_dir=TRAIN_DIR, split='val', augment=False, target_size=TARGET_SIZE
)

# Create data loaders with appropriate batch size
BATCH_SIZE = 16 if HAS_TPU or torch.cuda.is_available() else 4
NUM_WORKERS = 4 if HAS_TPU or torch.cuda.is_available() else 2

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
    num_workers=NUM_WORKERS, pin_memory=True
)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
    num_workers=NUM_WORKERS, pin_memory=True
)

print(f"Data loaders created: {len(train_loader)} train, {len(val_loader)} val batches")
print(f"Using batch size: {BATCH_SIZE}, Workers: {NUM_WORKERS}")

# =============================================================================
# PART 8: TRAINING WITH IMPROVED LOSS AND METRICS
# =============================================================================

def laplace_log_likelihood(y_true, y_pred, sigma, sigma_min=70):
    """Compute Laplace Log Likelihood with sigma clipping"""
    sigma = np.maximum(sigma, sigma_min)
    delta = np.abs(y_true - y_pred)
    return -np.sqrt(2) * delta / sigma - np.log(np.sqrt(2) * sigma)

class ImprovedTrainer:
    def __init__(self, model, device, lr=1e-4):
        self.model = model
        self.device = device
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', patience=5, factor=0.5, verbose=True
        )
        self.best_loss = float('inf')
        self.scaler = GradScaler()
        
        # For TPU
        self.is_tpu = hasattr(device, 'type') and device.type == 'xla'
    
    def gaussian_nll_loss(self, mean_pred, log_var, targets):
        """Gaussian negative log likelihood loss"""
        return 0.5 * (torch.mean(torch.exp(-log_var) * (mean_pred - targets)**2 + log_var))
    
    def compute_metrics(self, mean_pred, log_var, targets):
        """Compute various metrics for evaluation"""
        # Convert to numpy for metric calculation
        mean_pred_np = mean_pred.detach().cpu().numpy()
        log_var_np = log_var.detach().cpu().numpy()
        targets_np = targets.detach().cpu().numpy()
        
        # Calculate sigma
        sigma_np = np.sqrt(np.exp(log_var_np))
        
        # Metrics
        mse = mean_squared_error(targets_np, mean_pred_np)
        rmse = np.sqrt(mse)
        r2 = r2_score(targets_np, mean_pred_np)
        
        # Laplace Log Likelihood
        lll = np.mean(laplace_log_likelihood(targets_np, mean_pred_np, sigma_np))
        
        return {
            'mse': mse,
            'rmse': rmse,
            'r2': r2,
            'lll': lll
        }
    
    def train_epoch(self, loader):
        self.model.train()
        total_loss = 0
        all_metrics = {'mse': 0, 'rmse': 0, 'r2': 0, 'lll': 0}
        
        for images, tabular, targets, _ in tqdm(loader, desc="Training"):
            if self.is_tpu:
                images, tabular, targets = images.to(self.device), tabular.to(self.device), targets.to(self.device)
            else:
                images, tabular, targets = images.to(self.device, non_blocking=True), \
                                         tabular.to(self.device, non_blocking=True), \
                                         targets.to(self.device, non_blocking=True)
            
            self.optimizer.zero_grad()
            
            if self.is_tpu:
                # TPU doesn't support AMP, use regular training
                mean_pred, log_var = self.model(images, tabular)
                loss = self.gaussian_nll_loss(mean_pred, log_var, targets)
                loss.backward()
                xm.optimizer_step(self.optimizer)
            else:
                # Use AMP for GPU
                with autocast():
                    mean_pred, log_var = self.model(images, tabular)
                    loss = self.gaussian_nll_loss(mean_pred, log_var, targets)
                
                self.scaler.scale(loss).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
            
            total_loss += loss.item()
            
            # Compute metrics
            metrics = self.compute_metrics(mean_pred, log_var, targets)
            for k in all_metrics:
                all_metrics[k] += metrics[k]
        
        # Average metrics
        for k in all_metrics:
            all_metrics[k] /= len(loader)
        
        return total_loss / len(loader), all_metrics
    
    def validate(self, loader):
        self.model.eval()
        total_loss = 0
        all_metrics = {'mse': 0, 'rmse': 0, 'r2': 0, 'lll': 0}
        
        with torch.no_grad():
            for images, tabular, targets, _ in tqdm(loader, desc="Validation"):
                if self.is_tpu:
                    images, tabular, targets = images.to(self.device), tabular.to(self.device), targets.to(self.device)
                else:
                    images, tabular, targets = images.to(self.device, non_blocking=True), \
                                             tabular.to(self.device, non_blocking=True), \
                                             targets.to(self.device, non_blocking=True)
                
                mean_pred, log_var = self.model(images, tabular)
                loss = self.gaussian_nll_loss(mean_pred, log_var, targets)
                total_loss += loss.item()
                
                # Compute metrics
                metrics = self.compute_metrics(mean_pred, log_var, targets)
                for k in all_metrics:
                    all_metrics[k] += metrics[k]
        
        # Average metrics
        for k in all_metrics:
            all_metrics[k] /= len(loader)
        
        return total_loss / len(loader), all_metrics
    
    def train(self, train_loader, val_loader, epochs=20):
        for epoch in range(epochs):
            train_loss, train_metrics = self.train_epoch(train_loader)
            val_loss, val_metrics = self.validate(val_loader)
            
            self.scheduler.step(val_loss)
            
            print(f"\nEpoch {epoch+1}/{epochs}:")
            print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
            print("Train Metrics - MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, LLL: {lll:.4f}".format(**train_metrics))
            print("Val Metrics   - MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, LLL: {lll:.4f}".format(**val_metrics))
            
            if val_loss < self.best_loss:
                self.best_loss = val_loss
                if self.is_tpu:
                    xm.save(model.state_dict(), 'best_model.pth')
                else:
                    torch.save(model.state_dict(), 'best_model.pth')
                print("✅ New best model saved!")




Using GPU accelerator
Pulmonary Fibrosis Progression Analysis - OPTIMIZED VERSION
Device: cuda
Train: 1549 rows, Test: 5 rows

Train data sample:
                     Patient  Weeks   FVC    Percent  Age   Sex SmokingStatus
0  ID00007637202177411956430     -4  2315  58.253649   79  Male     Ex-smoker
1  ID00007637202177411956430      5  2214  55.712129   79  Male     Ex-smoker
2  ID00007637202177411956430      7  2061  51.862104   79  Male     Ex-smoker
3  ID00007637202177411956430      9  2144  53.950679   79  Male     Ex-smoker
4  ID00007637202177411956430     11  2069  52.063412   79  Male     Ex-smoker

Train data statistics:
             Weeks          FVC      Percent          Age
count  1549.000000  1549.000000  1549.000000  1549.000000
mean     31.861846  2690.479019    77.672654    67.188509
std      23.247550   832.770959    19.823261     7.057395
min      -5.000000   827.000000    28.877577    49.000000
25%      12.000000  2109.000000    62.832700    63.000000
50%      28.00

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


Processed 176 patients with enhanced features


100%|██████████| 20.5M/20.5M [00:00<00:00, 166MB/s]


Model parameters: 6,183,462
Train patients: 140, Validation patients: 36
Dataset train: 140 valid patients
Dataset val: 36 valid patients
Data loaders created: 35 train, 3 val batches
Using batch size: 16, Workers: 4


In [6]:
# Start training
print("🚀 Starting training...")
trainer = ImprovedTrainer(model, DEVICE, lr=1e-4)
trainer.train(train_loader, val_loader, epochs=15)

print("🎯 Training completed!")


🚀 Starting training...


Validation: 100%|██████████| 3/3 [00:00<00:00,  4.77it/s]



Epoch 1/15:
Train Loss: 20.8499, Val Loss: 8.3679
Train Metrics - MSE: 55.3575, RMSE: 7.1712, R²: -0.8363, LLL: -4.7058
Val Metrics   - MSE: 58.8569, RMSE: 7.2804, R²: -1.0385, LLL: -4.7073
✅ New best model saved!


Validation: 100%|██████████| 3/3 [00:00<00:00,  6.11it/s]



Epoch 2/15:
Train Loss: 4.9857, Val Loss: 2.4407
Train Metrics - MSE: 52.9374, RMSE: 7.1170, R²: -0.6516, LLL: -4.7031
Val Metrics   - MSE: 53.6437, RMSE: 6.9135, R²: -0.8032, LLL: -4.6992
✅ New best model saved!


...


Epoch 13/15:
Train Loss: 1.7908, Val Loss: 3.0885
Train Metrics - MSE: 29.2810, RMSE: 5.2203, R²: 0.1500, LLL: -4.6655
Val Metrics   - MSE: 36.2617, RMSE: 5.5219, R²: -0.0448, LLL: -4.6748


Validation: 100%|██████████| 3/3 [00:00<00:00,  6.25it/s]



Epoch 14/15:
Train Loss: 1.7239, Val Loss: 3.1881
Train Metrics - MSE: 28.8702, RMSE: 5.2189, R²: 0.1030, LLL: -4.6640
Val Metrics   - MSE: 36.1208, RMSE: 5.5065, R²: -0.0344, LLL: -4.6751


Validation: 100%|██████████| 3/3 [00:00<00:00,  6.05it/s]


Epoch 15/15:
Train Loss: 1.7162, Val Loss: 3.0900
Train Metrics - MSE: 30.6464, RMSE: 5.3004, R²: 0.0514, LLL: -4.6643
Val Metrics   - MSE: 34.9573, RMSE: 5.4035, R²: 0.0092, LLL: -4.6732
🎯 Training completed!


In [7]:
# =============================================================================
# PART 9: BASELINE MODEL (LightGBM)
# =============================================================================

print("Training LightGBM baseline for comparison...")

# Prepare data for LightGBM
X = []
y = []
groups = []

for patient in P:
    features = TAB[patient]
    X.append(features)
    y.append(A[patient])
    groups.append(patient)

X = np.array(X)
y = np.array(y)

# Train LightGBM model
lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X, y)

# Evaluate
lgb_preds = lgb_model.predict(X)
lgb_mae = np.mean(np.abs(lgb_preds - y))
lgb_mse = mean_squared_error(y, lgb_preds)
lgb_rmse = np.sqrt(lgb_mse)
lgb_r2 = r2_score(y, lgb_preds)

print(f"LightGBM Baseline - MAE: {lgb_mae:.4f}, MSE: {lgb_mse:.4f}, RMSE: {lgb_rmse:.4f}, R²: {lgb_r2:.4f}")

# =============================================================================
# PART 10: IMPROVED SUBMISSION GENERATION
# =============================================================================

def create_improved_submission(model, test_dir, output_file='submission.csv'):
    """Create improved submission file with proper uncertainty handling"""
    print("Creating improved submission...")
    
    # Load test data
    test_df = pd.read_csv(DATA_DIR / 'test.csv')
    submissions = []
    
    model.eval()
    augmentor = MedicalAugmentation(augment=False, target_size=TARGET_SIZE)
    
    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        patient_id = row['Patient']
        weeks = row['Weeks']
        
        try:
            patient_dir = Path(test_dir) / patient_id
            
            # Load 2.5D image
            img = load_three_slices(patient_dir, target_size=TARGET_SIZE)
            if img is None:
                raise ValueError("No DICOM files found")
                
            img_tensor = augmentor(img).unsqueeze(0).to(DEVICE)
            
            # Get tabular features
            tab_features = get_enhanced_tabular_features(patient_id, row)
            tab_tensor = torch.tensor(tab_features).float().unsqueeze(0).to(DEVICE)
            
            with torch.no_grad():
                mean_pred, log_var = model(img_tensor, tab_tensor)
                fvc_pred = mean_pred.item()
                sigma = np.sqrt(np.exp(log_var.item()))
                
                # Apply competition sigma floor (70) only at submission time
                confidence = max(sigma, 70.0)
            
            # For each required week in the test set
            patient_week = f"{patient_id}_{weeks}"
            
            # Use the model's prediction directly (no slope adjustment)
            submissions.append({
                'Patient_Week': patient_week,
                'FVC': fvc_pred,
                'Confidence': confidence
            })
                
        except Exception as e:
            # Fallback to baseline prediction
            if patient_id in TAB:
                tab_features = TAB[patient_id]
                fvc_pred = lgb_model.predict(tab_features.reshape(1, -1))[0]
            else:
                fvc_pred = 2500  # Average FVC
                
            submissions.append({
                'Patient_Week': f"{patient_id}_{weeks}",
                'FVC': fvc_pred,
                'Confidence': 200.0  # Conservative uncertainty
            })
    
    # Create submission file
    submission_df = pd.DataFrame(submissions)
    submission_df.to_csv(output_file, index=False)
    print(f"Submission saved to {output_file}")
    return submission_df

# Generate submission
if test_df is not None and TEST_DIR.exists():
    submission = create_improved_submission(model, TEST_DIR, 'submission.csv')
    print("✅ Submission ready!")
    print(submission.head())
else:
    print("No test data found - skipping submission")

print("🎉 All done!")

Training LightGBM baseline for comparison...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000076 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 308
[LightGBM] [Info] Number of data points in the train set: 176, number of used features: 9
[LightGBM] [Info] Start training from score -4.524301
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

100%|██████████| 5/5 [00:00<00:00, 28.19it/s]

Submission saved to submission.csv
✅ Submission ready!
                   Patient_Week       FVC  Confidence
0   ID00419637202311204720264_6 -2.044669       200.0
1  ID00421637202311550012437_15 -2.059186       200.0
2   ID00422637202311677017371_6 -4.415662       200.0
3  ID00423637202312137826377_17 -9.395618       200.0
4   ID00426637202313170790466_0 -0.821343       200.0
🎉 All done!
